<p style="font-family: Arial; font-size:3em;color:black;"> Lab Exercise 10</p>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(5)  # creating pseudo-random numbers for reproducibility

In [2]:
# Let's consider the following A_orig matrix:

A_orig = np.array([[4, 2, 4, 5, 4, 2, 2], [4, 4, 3, 2, 2, 5, 4], [4, 1, 4, 7, 3, 6, 2], [8 ,1, 2, 0, 5, 0, 7], [4 , 5, 8, 7, 6, 2, 3]],dtype=float)
M, N = 5, 7
print (pd.DataFrame(A_orig).head())

     0    1    2    3    4    5    6
0  4.0  2.0  4.0  5.0  4.0  2.0  2.0
1  4.0  4.0  3.0  2.0  2.0  5.0  4.0
2  4.0  1.0  4.0  7.0  3.0  6.0  2.0
3  8.0  1.0  2.0  0.0  5.0  0.0  7.0
4  4.0  5.0  8.0  7.0  6.0  2.0  3.0


In [3]:
# Let's add some NaN(s) to  A_orig matrix

A = A_orig.copy()
A[3][1] = np.NaN
A[4][6] = np.NaN

A_df = pd.DataFrame(A)
print (A_df.head())

     0    1    2    3    4    5    6
0  4.0  2.0  4.0  5.0  4.0  2.0  2.0
1  4.0  4.0  3.0  2.0  2.0  5.0  4.0
2  4.0  1.0  4.0  7.0  3.0  6.0  2.0
3  8.0  NaN  2.0  0.0  5.0  0.0  7.0
4  4.0  5.0  8.0  7.0  6.0  2.0  NaN


In [285]:
# Fine tune K value such that the (average percentage) error for the 2 missing elements is minimum.
# Remember: this is an exercise where we have the actual values for missing elements. 
# In real-life scenarios missins elements are not known and you may need different metric(s)
# to assess the quality of your reconstructed matrix.

K = 1

P = np.abs(np.random.uniform(low=0, high=8, size=(M, K)))
Q = np.abs(np.random.uniform(low=0, high=8, size=(K, N)))
P = np.divide(P, K*P.max())
Q = np.divide(Q, K*Q.max())

In [286]:
def matrix_factorization(Rating_Matrix, P, Q, K, steps, alpha=0.001, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(len(Rating_Matrix)):
            for j in range(len(Rating_Matrix[i])):
                if ~np.isnan(Rating_Matrix[i][j]):
                    eij = Rating_Matrix[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eRating_Matrix = np.dot(P,Q)
        e = 0
        for i in range(len(Rating_Matrix)):
            for j in range(len(Rating_Matrix[i])):
                if ~np.isnan(Rating_Matrix[i][j]):
                    e = e + pow(Rating_Matrix[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        print("Total error at step", step, "is", e)
        if e < 0.0001:
            break
    return P, Q.T


In [289]:
matrix_factorization(A, P, Q.T, K, steps = 1000)
eA = np.matmul(eP, eQ.T) # returns estimated Rating_Matrix
print(A,'\n')
print(eA)

# Calculating average percentage error for the 2 missing elements
err1 = abs((eA[3][1] - A_orig[3][1]) / A_orig[3][1])
err2 = abs((eA[4][6] - A_orig[4][6]) / A_orig[4][6])

avg_err = (err1 + err2) / 2

Total error at step 0 is 116.9140939162006
Total error at step 1 is 116.91408294144895
Total error at step 2 is 116.91407198916846
Total error at step 3 is 116.91406105934443
Total error at step 4 is 116.91405015196231
Total error at step 5 is 116.91403926700734
Total error at step 6 is 116.91402840446486
Total error at step 7 is 116.91401756432023
Total error at step 8 is 116.91400674655887
Total error at step 9 is 116.91399595116606
Total error at step 10 is 116.91398517812725
Total error at step 11 is 116.91397442742782
Total error at step 12 is 116.91396369905318
Total error at step 13 is 116.91395299298863
Total error at step 14 is 116.9139423092197
Total error at step 15 is 116.9139316477318
Total error at step 16 is 116.91392100851027
Total error at step 17 is 116.91391039154061
Total error at step 18 is 116.91389979680822
Total error at step 19 is 116.91388922429867
Total error at step 20 is 116.91387867399736
Total error at step 21 is 116.91386814588967
Total error at step 22 

In [290]:
print("Best value of K:", K)
print("Average Percentage Error:", avg_err)

Best value of K: 1
Average Percentage Error: 2.3980790157858327
